In [ ]:
from constants import FINAL_CSV, CORE_FORMATTED_CSV, JSON_PATH
import pandas as pd
import json
import missingno as msno

In [ ]:
df = pd.read_csv(FINAL_CSV)

In [ ]:
df = df[df["id"] != "ID7HeZrU"]
df.drop(["url", "path", "id"], axis=1, inplace=True)

df = df.rename(
    columns={
        "unique_id": "id",
        "audio si tehnologie": "audio & technology",
        "confort si echipamente optionale": "comfort & optional equipment",
        "electronice si sisteme de asistenta": "electronics & assistance systems",
        "performanta": "performance",
        "siguranta": "safety",
        "vehicule electrice": "electric vehicles",
        "oferit de": "sold by",
        "are vin (serie sasiu)": "has vin (chassis number)",
        "marca": "manufacturer",
        "model": "model",
        "versiune": "version",
        "anul producției": "year",
        "km": "km",
        "combustibil": "fuel",
        "putere": "power",
        "capacitate cilindrica": "engine_capacity",
        "transmisie": "transmission",
        "norma de poluare": "pollution norm",
        "consum extraurban": "extra-urban consumption",
        "cutie de viteze": "gearbox",
        "consum urban": "urban consumption",
        "tip caroserie": "chassis",
        "emisii co2": "co2 emissions",
        "numar de portiere": "doors",
        "culoare": "color",
        "optiuni culoare": "color options",
        "numar locuri": "seats",
        "tara de origine": "country of origin",
        "data primei inmatriculari": "first registration date",
        "inmatriculat": "registered",
        "primul proprietar (de nou)": "first owner",
        "fara accident in istoric": "undamaged history",
        "carte de service": "service book",
        "stare": "condition",
        "description": "description",
        "price": "price",
        "currency": "currency",
        "generatie": "generation",
        "masina de epoca": "vintage car",
        "se emite factura": "invoice issued",
        "eligibil pentru finantare": "eligible for financing",
        "autonomie": "autonomy",
        "capacitate baterie": "battery capacity",
        "predare leasing": "leasing transfer",
        "consum mixt": "mixed consumption",
        "consum mediu": "average consumption",
        "contract baterie": "battery contract",
        "tuning": "tuning",
        "volan pe dreapta": "right hand drive",
        "timp de incarcare": "charging time",
        "garantie dealer (inclusa in pret)": "dealer warranty (included in price)",
        "sau in limita a": "or within",
        "garantie de la producator pana la": "manufacturer warranty until",
        "plata initiala (la predare)": "initial payment (on delivery)",
        "valoare rata lunara": "monthly rate value",
        "numar de rate lunare ramase": "number of remaining monthly rates",
        "valoare reziduala": "residual value",
    }
)

df.head(5)

## Remove electric vehicles and their specific columns as they are outliers

In [ ]:
ELECTRIC_COLUMNS = [
    "electric vehicles",
    "autonomy",
    "battery capacity",
    "battery contract",
    "charging time",
]


def remove_electric_vehicles(df):
    initial_len = len(df)
    df = df[df["fuel"] != "electric"]
    # for col in ELECTRIC_COLUMNS:
    #     df = df[df[col].isna()]
    print(f"Removed {initial_len - len(df)} electric vehicles")
    df = df.drop(
        ELECTRIC_COLUMNS,
        axis=1,
    )
    return df


print(df["fuel"].value_counts())
print(df["fuel"].unique())
df = remove_electric_vehicles(df)

## Remove vehicles that are in leasing, as their price is not accurate to our needs

In [ ]:
LEASING_COLUMNS = [
    "leasing transfer",
    "initial payment (on delivery)",
    "monthly rate value",
    "number of remaining monthly rates",
    "residual value",
]


def remove_leasing(df):
    initial_len = len(df)
    df = df[df["leasing transfer"] != "da"]
    # for col in LEASING_COLUMNS:
    #     df = df[df[col].isna()]
    print(f"Removed {initial_len - len(df)} leasing vehicles")
    df = df.drop(
        LEASING_COLUMNS,
        axis=1,
    )
    return df


df = remove_leasing(df)

## Remove tuned cars as they are outliers, and also not detailed enough, as tuning can vary a lot

In [ ]:
def remove_tuning(df):
    initial_len = len(df)
    df = df[df["tuning"] != "da"]
    print(f"Removed {initial_len - len(df)} tuned vehicles")
    df = df.drop(["tuning"], axis=1)
    return df


print(df["tuning"].value_counts())
print(df["tuning"].unique())
df = remove_tuning(df)

## Drop rows that have currency different than eur

In [ ]:
def remove_non_eur_currency(df):
    initial_len = len(df)
    df = df[df["currency"] == "eur"]
    print(f"Removed {initial_len - len(df)} non eur currency vehicles")
    df = df.drop(["currency"], axis=1)
    return df


print(df["currency"].isna().sum())
print(df["currency"].value_counts())
print(df["currency"].unique())
df = remove_non_eur_currency(df)

## Drop vintage cars, as they are outliers (also remove cars older than 2000)

In [ ]:
def remove_vintage_cars(df):
    initial_len = len(df)
    df = df[df["vintage car"] != "da"]
    print(f"Removed {initial_len - len(df)} vintage cars")
    df = df.drop(["vintage car"], axis=1)
    initial_len = len(df)
    df = df[df["year"] >= 2000]
    print(f"Removed {initial_len - len(df)} older than 2000 cars")
    return df


print(df["vintage car"].value_counts())
print(df["vintage car"].unique())
print(df["year"].isna().sum())
print(df["year"].value_counts())
print(df["year"].unique())
df = remove_vintage_cars(df)

## Drop the column "right hand drive" as there is a few data points for it, outliers

In [ ]:
def remove_volan_pe_dreapta(df):
    initial_len = len(df)
    df = df[df["right hand drive"] != "da"]
    print(f"Removed {initial_len - len(df)} right hand drive cars")
    df = df.drop(["right hand drive"], axis=1)
    return df


print(df["right hand drive"].value_counts())
print(df["right hand drive"].unique())
df = remove_volan_pe_dreapta(df)

## Remove fuel outliers

In [ ]:
print(df["fuel"].value_counts())


def remove_fuel_outliers(df):
    initial_len = len(df)
    df = df[df["fuel"].isin(["benzina", "diesel"])]
    df["fuel"] = df["fuel"].replace({"benzina": "gasoline", "diesel": "diesel"})
    print(f"Removed {initial_len - len(df)} fuel outliers")
    return df


df = remove_fuel_outliers(df)
print(df["fuel"].isna().sum())
print(df["fuel"].value_counts())
print(df["fuel"].unique())

## Format numeric columns

In [ ]:
def format_numeric_columns(df):
    def format_km():
        df["km"] = df["km"].str.replace(" km", "").str.replace(" ", "").astype(int)

    def format_horsepower():
        df["power"] = df["power"].str.replace(" cp", "").str.replace(" ", "").astype(int)

    def format_motor_capacity():
        df["engine_capacity"] = df["engine_capacity"].str.replace(" cm3", "").str.replace(" ", "").astype(int)

    def format_price():
        df["price"] = df["price"].astype(str)
        df["price"] = df["price"].str.replace(r"\..*", "", regex=True)
        df["price"] = df["price"].str.replace(r",.*", "", regex=True)
        df["price"] = df["price"].astype(int)

    format_km()
    format_horsepower()
    format_motor_capacity()
    format_price()
    return df


print(df["km"].isna().sum())
print(df["power"].isna().sum())
print(df["engine_capacity"].isna().sum())
print(df["price"].isna().sum())

df.loc[df["condition"] == "nou", "km"] = "1 km"
df = df.dropna(subset=["km", "power", "engine_capacity", "price"])

print(df["km"].isna().sum())
print(df["power"].isna().sum())
print(df["engine_capacity"].isna().sum())
print(df["price"].isna().sum())

df = format_numeric_columns(df)

## Drop consumption related columns as there is not enough data

In [ ]:
CONSUM_COLS = ["extra-urban consumption", "urban consumption", "mixed consumption", "average consumption"]

df = df.drop(columns=CONSUM_COLS)

## Drop warranty columns as there is not enough data

In [ ]:
WARRANTY_COLS = ["dealer warranty (included in price)", "or within", "manufacturer warranty until"]


def remove_warranty_vehicles(df):
    initial_len = len(df)
    for col in WARRANTY_COLS:
        df = df[df[col].isna()]
    print(f"Removed {initial_len - len(df)} warranty vehicles")
    df = df.drop(
        WARRANTY_COLS,
        axis=1,
    )
    return df


df = remove_warranty_vehicles(df)

In [ ]:
UNUSED_COLUMNS = [
    "invoice issued",
    "eligible for financing",
    "has vin (chassis number)",
]

MORE_UNUSED_COLUMNS = [
    "version",
    "generation",
    "country of origin",
    "color",
    "color options",
    "transmission",
]

HISTORY_COLS = [
    "first registration date",
    "registered",
    "first owner",
    "undamaged history",
    "service book",
]

POLUTION_COLS = ["pollution norm", "co2 emissions"]

OTHER_COLS = ["doors", "seats"]

df = df.drop(columns=UNUSED_COLUMNS)
df = df.drop(columns=MORE_UNUSED_COLUMNS)
df = df.drop(columns=HISTORY_COLS)
df = df.drop(columns=POLUTION_COLS)
df = df.drop(columns=OTHER_COLS)

## Concat custom options

In [ ]:
CUSTOM_OPTIONS_COLUMNS = [
    "audio & technology",
    "comfort & optional equipment",
    "electronics & assistance systems",
    "performance",
    "safety",
]

for col in CUSTOM_OPTIONS_COLUMNS:
    df[f"copy_{col}"] = df[col].copy()

COPY_COLUMNS = [f"copy_{col}" for col in CUSTOM_OPTIONS_COLUMNS]

for col in COPY_COLUMNS:
    print(df[col].dtype)
    print(df[col].isna().sum())
    df.fillna({col: ""}, inplace=True)

map_engl_to_ro = {
    "audio & technology": "audio si tehnologie",
    "comfort & optional equipment": "confort si echipamente optionale",
    "electronics & assistance systems": "electronice si sisteme de asistenta",
    "performance": "performanta",
    "safety": "siguranta",
}


def concatenate_custom_options(row):
    values = [f"{map_engl_to_ro[col.replace('copy_', '')]}: {row[col]}" for col in COPY_COLUMNS if row[col] != "" and row[col] != "nan"]
    if len(values) == 0:
        return None
    return "\n".join(values)


df["options"] = df.apply(concatenate_custom_options, axis=1)

df = df.drop(columns=CUSTOM_OPTIONS_COLUMNS)
df = df.drop(columns=COPY_COLUMNS)

## Replace sold by column with a boolean column

In [ ]:
print(df["sold by"].isna().sum())
print(df["sold by"].value_counts())
print(df["sold by"].unique())

df["sold_by_company"] = df["sold by"].apply(lambda x: True if x == "firma" else False)
df = df.drop(columns=["sold by"])

print(df["sold_by_company"].isna().sum())
print(df["sold_by_company"].value_counts())
print(df["sold_by_company"].unique())

## Drop the condition column as there are only 1731 rows that are new, and remove those rows as they are outliers

In [ ]:
initial_len = len(df)
df["condition"] = df["condition"].apply(lambda x: True if x == "nou" else False)
df = df[df["condition"] == False]
df = df.drop(columns=["condition"])
print(f"Removed {initial_len - len(df)} new cars")

## Format gearbox, drop the empty rows, make it a boolean column named is_automatic

In [ ]:
print(df["gearbox"].isna().sum())
print(df["gearbox"].value_counts())
print(df["gearbox"].unique())

df = df.dropna(subset=["gearbox"])
df["is_automatic"] = df["gearbox"].apply(lambda x: True if x == "automata" else False)
df = df.drop(columns=["gearbox"])

print(df["is_automatic"].isna().sum())
print(df["is_automatic"].value_counts())
print(df["is_automatic"].unique())

## Format id column

In [ ]:
initial_len = len(df)
df = df.dropna(subset=["id"])
print(f"Removed {initial_len - len(df)} rows with na id")

df["id"] = df["id"].astype(str)
df["id"] = df["id"].apply(lambda x: x.zfill(6))

print(df["id"].isna().sum())
print(df["id"].value_counts())

In [ ]:
msno.bar(df)

## Concat strings columns to a full description

In [ ]:
COLUMNS_TO_CONCAT = ["options", "description"]

for col in COLUMNS_TO_CONCAT:
    df[f"copy_{col}"] = df[col].copy()

COPY_COLUMNS = [f"copy_{col}" for col in COLUMNS_TO_CONCAT]

for col in COPY_COLUMNS:
    print(df[col].dtype)
    print(df[col].isna().sum())
    df.fillna({col: ""}, inplace=True)


def concatenate_full_description(row):
    values = [row[col] for col in COPY_COLUMNS if row[col] != "" and row[col] != "nan"]
    if len(values) == 0:
        return None
    return "\n".join(values)


df["input"] = df.apply(concatenate_full_description, axis=1)
df = df.drop(columns=COLUMNS_TO_CONCAT)
df = df.drop(columns=COPY_COLUMNS)
df = df.rename(columns={"input": "description"})

## In tip caroserie make masina mica, masina de oras to be compacta

In [ ]:
df = df.replace({"chassis": {"masina mica": "compacta", "masina de oras": "compacta"}})
initial_len = len(df)
df = df[df["chassis"] != "cabrio"]
print(f"Removed {initial_len - len(df)} cabrio cars")

In [ ]:
msno.bar(df)

In [ ]:
REQUIRED_COLUMNS = [
    "manufacturer",
    "model",
    "year",
    "fuel",
    "km",
    "power",
    "engine_capacity",
    "chassis",
    "price",
    "id",
    "sold_by_company",
    "description",
    "is_automatic",
]

initial_len = len(df)
df = df.dropna(subset=REQUIRED_COLUMNS)
print(f"Removed {initial_len - len(df)} rows with na in required columns")

df = df[
    [
        "id",
        "price",
        "manufacturer",
        "model",
        "year",
        "km",
        "power",
        "engine_capacity",
        "fuel",
        "chassis",
        "is_automatic",
        "sold_by_company",
        "description",
    ]
]

df["year"] = df["year"].astype(int)

## OUTLIERS

In [ ]:
# drop the rows where power > 650 or  power < 50
initial_len = len(df)
df = df[df["power"] <= 600]
df = df[df["power"] >= 50]
print(f"power: removed {initial_len - len(df)} rows.")

# drop rows with engine_capacity < 500 or engine_capacity > 4000
initial_len = len(df)
df = df[df["engine_capacity"] >= 500]
df = df[df["engine_capacity"] <= 4000]
print(f"engine_capacity:  removed {initial_len - len(df)} rows.")

# drop rows with price > 40_000
initial_len = len(df)
df = df[df["price"] <= 40_000]
print(f"price: removed {initial_len - len(df)} rows.")

# drop rows with km > 500_000
initial_len = len(df)
df = df[df["km"] <= 500_000]
print(f"km: removed {initial_len - len(df)} rows.")

initial_len = len(df)
temp_df = df["manufacturer"].value_counts()
df = df[df["manufacturer"].isin(temp_df[temp_df >= 100].index)]
print(f"manufacturer: removed {initial_len - len(df)} rows.")

In [ ]:
msno.bar(df)

In [ ]:
df.to_csv(CORE_FORMATTED_CSV, index=False)

In [ ]:
categorical_columns = ["fuel", "chassis"]
categorical_values = {column: df[column].unique().tolist() for column in categorical_columns}

manufacturer_and_models = df.groupby("manufacturer")["model"].apply(lambda x: list(set(x))).to_dict()

json_output = {
    "manufacturerAndModels": manufacturer_and_models,
    "fuel": categorical_values["fuel"],
    "chassis": categorical_values["chassis"],
}

with open(JSON_PATH, "w") as f:
    json.dump(json_output, f)